In [ ]:
# Подготовка
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve

import zipfile


__z = zipfile.ZipFile("dataset.zip")
#df_str = __z.open("Vehicle_policies_2020.csv").read().decode("utf-8")
df_str_fs = __z.open("Vehicle_policies_2020.csv")

df_src = pd.read_csv(df_str_fs)


# Предварительная обработка данных

Данные [отсюда](https://www.kaggle.com/datasets/lakshmanraj/vehicle-insurance-policy):

~~1. pol_number - Номер полюса~~

~~2. pol_eff_dt - Дата вступления в силу полиса автострахования~~

3. gender - Пол водителя: F, M

4. agecat - Возрастная категория водителя: 1, 2, 3, 4, 5, 6 (1 - моложе, ..., 6 - старше)

~~5. date_of_birth - Дата рождения водителя~~

6. credit_score - Кредитный рейтинг водителя: от 301=плохо до 850=прекрасно

7. area - Категория: A, B, C, D, E, F

8. traffic_index - Дорожный индекс района проживания водителя:

- 100 = среднее по стране

- Больше 100 = плохие условия движения

- Меньше 100 = хорошие условия движения

9. veh_age - Возраст транспортного средства: 1, 2, 3, 4 (1 - новее, ..., 4 - старее)

10. veh_body - Кузов транспортного средства, coded as:
- BUS
- CONVT = convertible
- COUPE
- HBACK = hatchback
- HDTOP = hardtop
- MCARA = motorized caravan
- MIBUS = minibus
- PANVN = panel van
- RDSTR = roadster
- STNWG = station wagon
- TRUCK
- UTE = utility

11. veh_value - Стоимость транспортного средства, в $10 000

~~12. months_insured - Количество месяцев, на которые приобретена страховка транспортного средства~~

~~13. claim_office - Office location of claim handling agent: A, B, C, D~~

14. numclaims - Кол-во претензий: 0=нет_претензий

15. claimcst0 - Стоимость претензий: 0=нет_претензий

~~16. annual_premium - Total charged premium i.e. the cost of insurance~~

In [ ]:
# Look dataframe
pd.set_option('display.max_columns', df_src.shape[1])
df_src.head(300)

In [ ]:
# Look dataframe
#df_src.head()
pd.set_option('display.max_columns', df_src.shape[1])
df_src.tail(300)

# Проектирование признаков

## Убираем дубликаты

In [ ]:
print(f"Number of (rows, columns): {df_src.shape}")
duplicate_rows_df = df_src[df_src.duplicated()]
print(f"Number of duplicate (rows, columns): {duplicate_rows_df.shape}")
df_src = df_src.drop_duplicates()
print(f"Number of (rows, columns) after drop dublicates: {df_src.shape}")

## Смотрим, чтобы не было нерелевантных данных

### Как поступить с такими данными?

![](./imgs/if_missing_data.png)


In [ ]:
pd.set_option('display.max_rows', df_src.shape[1])
df_src.count() # Кол-во не None значений в каждой колонке

In [ ]:
pd.set_option('display.max_rows', df_src.shape[1])
print(df_src.isnull().sum()) # Смотрим есть ли null хотя бы в каком-нибудь столбце


In [ ]:
# Удалим посностью claim_office
del df_src["claim_office"]

# Удалим date_of_birth
del df_src["date_of_birth"]

# Удалим pol_number
del df_src["pol_number"]

# Удалим null'ы
df_src = df_src.dropna()

# Удалим дубликаты, если они появились после удаления столбцов
df_src = df_src.drop_duplicates()

print(f"\nNumber of (rows, columns): {df_src.shape}")

print("\nКол-во не None значений:")
print(df_src.count())

print("\nКол-во null\'ов:")
print(df_src.isnull().sum())

In [ ]:
plt.figure(figsize=(100,50))

df_src.hist(column="annual_premium")

print(f"Range of annual_premium: [{df_src['annual_premium'].min()} - {df_src['annual_premium'].max()}]")


In [ ]:
df_date_check = df_src.copy()

basedate = pd.Timestamp("2022-11-28")
# basedate = pd.Timestamp("11/28/2022") # То что в столбце "pol_eff_dt", тип str
            # pd.Timestamp("2022-11-28") and pd.Timestamp("11/28/2022") are equals

df_date_check["time_since_in_days"] = df_date_check["pol_eff_dt"].apply(lambda x: (basedate - pd.Timestamp(x)).days)


#df_date_check.hist(column="time_since_in_days")
print(f"Range of pol_eff_dt: [{df_date_check['time_since_in_days'].min()} - {df_date_check['time_since_in_days'].max()}]")


In [ ]:
# Никчёмные столбецы
del df_src["annual_premium"]
del df_src["pol_eff_dt"]
           
df_src = df_src.drop_duplicates()
           
print(f"\nNumber of (rows, columns): {df_src.shape}")

### В итоге будут убраны столбцы:

- `claim_office`: По большей части - это `null`'ы. 

- `date_of_birth`: Зачем это нужно, если есть `agecat`. 

- `pol_number`: Номер полюса никак не влияет. 

- `annual_premium`: Одно и тоже число. 

- `pol_eff_dt`: Дата оформления полюса не влияет на кластеризацию. Плюс ко всем, здесь период всего лишь 363 дня. 


## Кодирование строк + float->int

In [ ]:
# Смотрим типы столбцов

pd.set_option('display.max_rows', df_src.shape[1])

nullout = '''
Должно быть:

pol_number          int64
pol_eff_dt         cat->int
gender             cat->int
agecat              int64
credit_score        int64
area               cat->int
traffic_index       int64
veh_age             int64
veh_body           cat->int
veh_value         float64
numclaims           int64
claimcst0         float64
'''

df_src.dtypes

In [ ]:
df_nums = df_src.copy()

# Инты там, где они нужны
ints_cols = ["agecat", "credit_score", "traffic_index"]
df_nums[ ints_cols ] = df_nums[ints_cols].astype(int)

# Категориальные:

# veh_body
df_nums["veh_body"] = df_nums["veh_body"].astype('category')
df_nums["veh_body"] = df_nums["veh_body"].cat.codes
df_nums["veh_body"] = df_nums["veh_body"].astype(int)

# Не veh_body
cat_code = {"gender": {"M": 0, "F": 1}, 
            "area": {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5}}
df_nums = df_nums.replace(cat_code)

pd.set_option('display.max_rows', df_src.shape[1])
df_nums.dtypes

## Нормализация

In [ ]:
df_nums_scaled = df_nums.copy()
df_nums_scaled = (df_nums-df_nums.min ())/(df_nums.max ()-df_nums.min ())
df_nums_scaled.head(300)


# Графики

![](./imgs/which_visualization.png)


In [ ]:
numeric_columns_of_interest = ["credit_score", "traffic_index", "veh_value", "numclaims", "claimcst0"]
for col_i in numeric_columns_of_interest:
    print(f"Range of {col_i}: [{df_src[col_i].min()} - {df_src[col_i].max()}]")

In [ ]:
# Тепловая карта

plt.figure(figsize=(25, 13))
c= df_nums.corr()
sns.heatmap(c, cmap="BrBG", annot=True)
c

In [ ]:
# Гистограммы

plt.figure(figsize=(10, 10))

cols = df_nums.columns
for col_i in cols:
    df_nums.hist(column=col_i)


# Кластеризация


In [ ]:
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.mixture import GaussianMixture as EM

from sklearn import metrics

from scipy.spatial.distance import pdist

from collections import defaultdict


RndState = 5051


## k-means

In [ ]:
Js = [] # inertia
N_max = 13

for i in range(1, N_max):
    kmeans = KMeans(n_clusters=i, random_state=RndState).fit(df_nums)
    Js.append(np.sqrt(kmeans.inertia_))

Ds = []
for i in range(1, len(Js)-1):
    Ds.append( abs(Js[i] - Js[i+1]) / abs(Js[i-1] - Js[i]) )
Ds = [round(Ds[i], 3) for i in range(len(Ds))]
print(f"Ds = {Ds}, \nmin = {min(Ds)} with k = {Ds.index(min(Ds))+1}")
    
plt.plot(range(1, N_max), Js, marker="s")
plt.xlabel("$k$")
plt.ylabel("$J(C_k)$")

In [ ]:
# Число кластеров 9 звучит хорошо
N_CLASTERS = 9
kmeans = KMeans(n_clusters=N_CLASTERS, random_state=RndState).fit(df_nums)

#print(f"kmeans.labels={len(kmeans.labels_)}, number of rows = {df_nums.shape[0]}")

d_classes = {i: 0 for i in range(N_CLASTERS)}
for i in kmeans.labels_:
    d_classes[i] += 1

print(f"Numbers of classes: {d_classes}")

pca = PCA(n_components=2)
res = pca.fit_transform(df_nums)
plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=kmeans.labels_, s=50, cmap='viridis')
plt.title('PCA')

print(f"\n=====\nSilhouette says: {metrics.silhouette_score(df_nums, kmeans.labels_)}\n=====") 
# From {-1} is super bad   --->   to {1} is super good

## Mean-Shift

In [ ]:

df = df_nums_scaled.copy()

for bandwidth_i_int in range(5, 100, 10):
    bandwidth_i = bandwidth_i_int / 100
    flat_shift = MeanShift(bandwidth=bandwidth_i).fit(df)
    number_of_class = len(set(flat_shift.labels_))
    print(f"\nFor bandwidth={bandwidth_i} {number_of_class} classes")
    if(number_of_class < 15):
        d = defaultdict(lambda:0)
        for row_i in flat_shift.labels_:
            d[row_i] += 1
        print(f"\t{d}")


In [ ]:
# Пусть bandwidth=
flat_shift = MeanShift(bandwidth=0.5).fit(df_nums_scaled)
d = defaultdict(lambda:0)
for row_i in flat_shift.labels_:
    d[row_i] += 1

print(f"Numbers of classes: {d}")

pca = PCA(n_components=2)
res = pca.fit_transform(df_nums_scaled)
plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=flat_shift.labels_, s=50, cmap='viridis')
plt.title('PCA')

print(f"\n=====\nSilhouette says: {metrics.silhouette_score(df_nums_scaled, flat_shift.labels_)}\n=====")
# From {-1} is super bad   --->   to {1} is super good

## DBSCAN

In [ ]:

df = df_nums_scaled.copy()

for eps_i_int in range(5, 50, 5):
    eps_i = eps_i_int / 100
    for min_samples_i in range(3, 9, 2):
        dbscan = DBSCAN(eps=eps_i, min_samples=min_samples_i).fit(df)
        number_of_class = len(set(dbscan.labels_))
        print(f"\nFor eps={eps_i} and min_samples={min_samples_i} {number_of_class} classes")
        if(number_of_class < 15):
            d = defaultdict(lambda:0)
            for row_i in dbscan.labels_:
                d[row_i] += 1
            print(f"\t{d}")
        

In [ ]:
# Пусть eps=0.3 and min_samples=7
dbscan = DBSCAN(eps=0.3, min_samples=7).fit(df_nums_scaled)
d = defaultdict(lambda:0)
for row_i in dbscan.labels_:
    d[row_i] += 1

print(f"Numbers of classes: {d}")

pca = PCA(n_components=2)
res = pca.fit_transform(df_nums_scaled)
plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=dbscan.labels_, s=50, cmap='viridis')
plt.title('PCA')

print(f"\n=====\nSilhouette says: {metrics.silhouette_score(df_nums_scaled, dbscan.labels_)}\n=====")
# From {-1} is super bad   --->   to {1} is super good

## Иерархическая кластеризация

In [ ]:
Js = [] # inertia
N_max = 13

for i in range(1, N_max):
    agglomerative = AgglomerativeClustering(n_clusters=i).fit(df_nums)
    Js.append(np.sqrt(agglomerative.inertia_))

Ds = []
for i in range(1, len(Js)-1):
    Ds.append( abs(Js[i] - Js[i+1]) / abs(Js[i-1] - Js[i]) )
Ds = [round(Ds[i], 3) for i in range(len(Ds))]
print(f"Ds = {Ds}, \nmin = {min(Ds)} with k = {Ds.index(min(Ds))+1}")
    
plt.plot(range(1, N_max), Js, marker="s")
plt.xlabel("$k$")
plt.ylabel("$J(C_k)$")


# Комп умер, GG

In [ ]:
# верхний треугольник матрицы попарных расстояний
distance_mat = pdist(df_nums)
    

Z = hierarchy.linkage(distance_mat, "complete")
            # linkage — реализация агломеративного алгоритма
    
plt.figure(figsize=(10, 5))
dn = hierarchy.dendrogram(Z, color_threshold=0.5)

# Комп умер, GG

## EM

In [ ]:
N_max = 13

for i in range(1, N_max):
    em = GaussianMixture(n_components=i, random_state=RndState).fit(df_nums_scaled)
    pd.set_option('display.max_rows', i)
    print(f"\n{pd.DataFrame(em.predict(df_nums_scaled)).value_counts()}")

In [ ]:
# Пусть n_components=9
N_CLASTERS = 9
em = GaussianMixture(n_components=N_CLASTERS, random_state=RndState).fit(df_nums_scaled)

em_labels = em.predict(df_nums_scaled)
d_classes = {i: 0 for i in range(N_CLASTERS)}
for i in em_labels:
    d_classes[i] += 1

print(f"Numbers of classes: {d_classes}")

pca = PCA(n_components=2)
res = pca.fit_transform(df_nums)
plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=em.predict(df_nums_scaled), s=50, cmap='viridis')
plt.title('PCA')

print(f"\n=====\nSilhouette says: {metrics.silhouette_score(df_nums_scaled, em_labels)}\n=====")
# From {-1} is super bad   --->   to {1} is super good